# NLP analysis of Zomato reviews

# **Step1:** Importing libraries and reading csv dataset

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import re # regular expression(re) - function to remove special characters

In [2]:
import nltk # importing natural language toolkit (nltk) library
nltk.download('punkt') # downloading puncutuation (punkt) from nltk library

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2211570\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('stopwords') # downloading stopwords from nltk library

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2211570\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Importing dataset
reviews = pd.read_csv("Zomato_reviews.csv")

In [5]:
reviews.head()

,rating,review_text
0,1.0,"Their service is worst, pricing in menu is dif..."
1,5.0,really appreciate their quality and timing . I...
2,4.0,"Went there on a Friday night, the place was su..."
3,4.0,A very decent place serving good food.\r\nOrde...
4,5.0,One of the BEST places for steaks in the city....


In [6]:
reviews.shape

(27762, 2)

# **Step 2:** Text cleaning / Preprocessing

In [7]:
reviews.describe(include="all")

,rating,review_text
count,27762.000000,27748
unique,NaN,10548
top,NaN,good
freq,NaN,278
mean,3.665784,NaN
std,1.284573,NaN
min,1.000000,NaN
25%,3.000000,NaN
50%,4.000000,NaN
75%,5.000000,NaN


In [8]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27762 entries, 0 to 27761
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rating       27762 non-null  float64
 1   review_text  27748 non-null  object 
dtypes: float64(1), object(1)
memory usage: 433.9+ KB


In [9]:
reviews.isnull().value_counts()

rating  review_text
False   False          27748
        True              14
dtype: int64

14 rows are missing the review text - need to get rid of these records

In [10]:
reviews1 = reviews[~reviews['review_text'].isnull()].copy()
reviews1.reset_index(inplace=True, drop=True)

In [11]:
reviews.shape, reviews1.shape

((27762, 2), (27748, 2))

In [12]:
reviews1['rating'].value_counts()

4.0    8631
5.0    8108
3.0    3761
1.0    3126
2.0    1675
3.5    1077
4.5     932
2.5     261
1.5     177
Name: rating, dtype: int64

## Converting to list for easy manipulation

In [13]:
reviews_list = reviews1['review_text'].values

In [14]:
reviews_list

array(['Their service is worst, pricing in menu is different from bill. They can give you a bill with increased pricing. Even for serving water,menu, order you need to call them 3-4 times even on a non busy day.',
       "really appreciate their quality and timing . I have tried the thattil kutti dosa I've been addicted to the dosa really and the chutney... really good and money worth much better than a thattukada must try it",
       'Went there on a Friday night, the place was surprisingly empty. Interesting menu which is almost fully made of dosas. I had bullseye dosa and cheese masala dosa. The bullseye Dosa was really good, with the egg perfectly cooked to a half boiled state. The masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. The chutney was good, the sambar was average. The dishes are reasonably priced.',
       ...,
       'Pizza is really thin crust and made from freshly prepared dough. Unlimited pizza plan is really good if you are giv

In [15]:
# to see the length of the reviews list
len(reviews_list)

27748

In [16]:
# Checking if a list contains an upper case letter
sum(rl.isupper() for rl in reviews_list) 

34

The reviews list contains `34 upppercase` letters.

## Text clean up / preprocessing

###  Normalize the case

Converting each word into its lower case because it's useless to have same words in different cases. Python considers same word is unique if it is written in both lower and upper case (eg., 'taste','TASTE','Taste',..)

In [17]:
reviews_lower = [txt.lower() for txt in reviews_list]

In [18]:
reviews_lower[6:9]

["this place ia for ultimate steak lovers!\r\nboth chicken and pork steaks are just so scrumptious and succulent! we tried honey mustard based steakcious and what's in a steak these two dishes there and both were so tasty! the mocktails are also quite good so nice place to do an awesome dine-out with family and friend!",
 "it's a shame if you haven't tried once upon a flame. :)\r\n\r\nlip smacking food. for the first time i licked off a sizzler plate. it was that bloody good.\r\n\r\ntwo of us ordered fish chilly , chicken steak sizzler and pork steak. the sauce was killer , meat was tender and the sides were absolutely fresh.\r\n\r\nyou end up having old salad sides and mashed potatoes at a lot of places but they served everything super fresh.\r\n\r\ni just wish this place runs a packed house.\r\n\r\nmy only complaint is their drinks were tad too sweet\r\n\r\nhighly recommended.",
 'we visited this place after we were tired and hungry after roaming around agra lake. it is a newly opene

In [19]:
sum(rl.isupper() for rl in reviews_lower)

0

Now, the list does not contain any upper case letters.

### Remove extra line breaks

Python recognizes "\r", "\n", and "\r\n" as line boundaries for 8-bit strings. 

Splitting the string with no separator arg, which will treat consecutive whitespace as a single separator (including newlines and tabs). Then join using space:

     eg., In : " ".join("\n\nsome    text \r\n with multiple whitespace".split())
          Out: 'some text with multiple whitespace'
        
          # without a whitespace ("".join())


In [20]:
reviews_lower = [" ".join(txt.split()) for txt in reviews_lower]

In [21]:
reviews_lower[6:9]

["this place ia for ultimate steak lovers! both chicken and pork steaks are just so scrumptious and succulent! we tried honey mustard based steakcious and what's in a steak these two dishes there and both were so tasty! the mocktails are also quite good so nice place to do an awesome dine-out with family and friend!",
 "it's a shame if you haven't tried once upon a flame. :) lip smacking food. for the first time i licked off a sizzler plate. it was that bloody good. two of us ordered fish chilly , chicken steak sizzler and pork steak. the sauce was killer , meat was tender and the sides were absolutely fresh. you end up having old salad sides and mashed potatoes at a lot of places but they served everything super fresh. i just wish this place runs a packed house. my only complaint is their drinks were tad too sweet highly recommended.",
 'we visited this place after we were tired and hungry after roaming around agra lake. it is a newly opened place and as the name suggests their specia

### Tokenize 

   * **Tokenization** is breaking the raw text into small chunks.
   * Breaking the sentece into separate words are called tokens.
   * Split words whenever there is a space between them.
   * Treating punctuation marks as separate tokens since punctuation also has meaning. 

There are various kinds of tokenization techniques available with different methods and libraries. Ref: https://towardsdatascience.com/tokenization-for-natural-language-processing-a179a891bad4

Here we use `word_tokenize` from `nltk.tokenize` library, which splits the words whenever there is a space and punctuation marks between them.

    eg., In: print(word_tokenize("protect every creatures, treated in a best."))
         Out: ['protect', 'every', 'creatures', ',', 'treated', 'to', 'the', 'best', '.']

In [22]:
from nltk.tokenize import word_tokenize

In [23]:
print(word_tokenize(reviews_lower[0]))

['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.']


In [24]:
reviews_tokens = [word_tokenize(sent) for sent in reviews_lower]
print(reviews_tokens[0:4])

[['their', 'service', 'is', 'worst', ',', 'pricing', 'in', 'menu', 'is', 'different', 'from', 'bill', '.', 'they', 'can', 'give', 'you', 'a', 'bill', 'with', 'increased', 'pricing', '.', 'even', 'for', 'serving', 'water', ',', 'menu', ',', 'order', 'you', 'need', 'to', 'call', 'them', '3-4', 'times', 'even', 'on', 'a', 'non', 'busy', 'day', '.'], ['really', 'appreciate', 'their', 'quality', 'and', 'timing', '.', 'i', 'have', 'tried', 'the', 'thattil', 'kutti', 'dosa', 'i', "'ve", 'been', 'addicted', 'to', 'the', 'dosa', 'really', 'and', 'the', 'chutney', '...', 'really', 'good', 'and', 'money', 'worth', 'much', 'better', 'than', 'a', 'thattukada', 'must', 'try', 'it'], ['went', 'there', 'on', 'a', 'friday', 'night', ',', 'the', 'place', 'was', 'surprisingly', 'empty', '.', 'interesting', 'menu', 'which', 'is', 'almost', 'fully', 'made', 'of', 'dosas', '.', 'i', 'had', 'bullseye', 'dosa', 'and', 'cheese', 'masala', 'dosa', '.', 'the', 'bullseye', 'dosa', 'was', 'really', 'good', ',', 'w

### Expanding contractions
**Contractions** are shortened version of words or syllables. They often exist in either written or spoken forms in the English language. These shortened versions or contractions of words are created by removing specific letters and sounds. Examples would be, `do not` to `don’t` and `I would` to `I’d`.

    eg., In: 1 |txt = ["he wouldn't come","I'd love to explore"]
             2 |exp_cont = [contractions.fix(word) for word in txt]
             3 |exp_cont
         Out: ['he would not come', 'I would love to explore']

> ***Expanding contractions because it reduces dimensionality and the redundancy in our data, or even help filter out stopwords. It makes our code computationally cheaper.***


In [25]:
# !pip install contractions

In [26]:
import contractions

In [27]:
reviews_contractions = [contractions.fix(cont) for cont in reviews_lower]
reviews_contractions[2:6]

['went there on a friday night, the place was surprisingly empty. interesting menu which is almost fully made of dosas. i had bullseye dosa and cheese masala dosa. the bullseye dosa was really good, with the egg perfectly cooked to a half boiled state. the masala in the cheese masala was good, but the cheese was a bit too chewy for my liking. the chutney was good, the sambar was average. the dishes are reasonably priced.',
 'a very decent place serving good food. ordered chilli fish, chicken & pork sizzler. everything tasted good but pork could have been slightly better cooked. tried 2 beverages, both were very sweet.',
 'one of the best places for steaks in the city. tried the beef steak with chili rum & grilled fish with orange and jalapenos. both were exceptionally good. the herbed rice and mashed potatoes serves alongside were equally delecatble. service is prompt and zomato gold is a great steal. if you are a steak lover, this place is a must visit. hope they come up with another 

### Remove Punctuations

Removal of punctuations is very helpful because it doesn't hold any vital information as that models are aware of the difference between hurray and hurray!.

In [28]:
import string

In [29]:
print(string.punctuation) # seeing punctuations present in string library

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [30]:
# removal of those puncuations inside the reviews_contractions list
stop_punct = []
for word in reviews_contractions:
    for letter in word:
        if letter in string.punctuation:
            word = word.replace(letter,"")
    stop_punct.append(word)
print(stop_punct[0:6])

['their service is worst pricing in menu is different from bill they can give you a bill with increased pricing even for serving watermenu order you need to call them 34 times even on a non busy day', 'really appreciate their quality and timing  i have tried the thattil kutti dosa i have been addicted to the dosa really and the chutney really good and money worth much better than a thattukada must try it', 'went there on a friday night the place was surprisingly empty interesting menu which is almost fully made of dosas i had bullseye dosa and cheese masala dosa the bullseye dosa was really good with the egg perfectly cooked to a half boiled state the masala in the cheese masala was good but the cheese was a bit too chewy for my liking the chutney was good the sambar was average the dishes are reasonably priced', 'a very decent place serving good food ordered chilli fish chicken  pork sizzler everything tasted good but pork could have been slightly better cooked tried 2 beverages both 

In [31]:
#stop_punct = [''.join(letter for letter in word if letter not in string.punctuation)
 #           for word in reviews_contractions]

### Regular Expression

**Regular expressions** or **RegEx** is defined as a sequence of characters that define pattern-matching, search-and-replace, and elimination functions. All type of noises can be removed with the help of regular expressions.

#### Removing special characters

**Special characters** are non-alphanumeric characters. These characters are most often found in comments, references, currency numbers, etc. These characters add no value to text-understanding and induce noise into algorithms. Thus, Regular expressions (regex) can be used to get rid of these characters and numbers.

In [32]:
stop_splchar = [(re.sub(r'[^a-zA-z., ]', r'', x)) for x in stop_punct]
print(stop_splchar[6:10])

['this place ia for ultimate steak lovers both chicken and pork steaks are just so scrumptious and succulent we tried honey mustard based steakcious and what is in a steak these two dishes there and both were so tasty the mocktails are also quite good so nice place to do an awesome dineout with family and friend', 'it is a shame if you have not tried once upon a flame  lip smacking food for the first time i licked off a sizzler plate it was that bloody good two of us ordered fish chilly  chicken steak sizzler and pork steak the sauce was killer  meat was tender and the sides were absolutely fresh you end up having old salad sides and mashed potatoes at a lot of places but they served everything super fresh i just wish this place runs a packed house my only complaint is their drinks were tad too sweet highly recommended', 'we visited this place after we were tired and hungry after roaming around agra lake it is a newly opened place and as the name suggests their specialty is barbecued a

In [33]:
# checking whether any special characters/punctuations is present
["``","....","=",'ã'] in stop_splchar

False

#### Remove Stopwords
The words which are generally filtered out before processing a natural language are called **stop words**. These are actually the most common words in any language (like articles, prepositions, pronouns, conjunctions, etc) and does not add much information to the text.

> Removal of few stop words is problematic because few of them are highly dependent on the task we are performing and the goal we want to achieve. For example, if we are training a model that can perform the sentiment analysis task (in twitter hashtags, reviews, etc.), we might not remove the stop words.

        Movie review: “The movie was not good at all.”
        Text after removal of stop words: “movie good”

we can clearly see that the review for the movie was negative. However, after the removal of stop words, the review became positive, which is not the reality. Thus, we are removing those words from original stop words list.

In [34]:
from nltk.corpus import stopwords

In [35]:
stop_words = stopwords.words("english")

In [36]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
len(stop_words)

179

In [38]:
stop_words.remove("no")
stop_words.remove("not")
stop_words.remove("don")
stop_words.remove("won")

In [39]:
len(stop_words)

175

In [40]:
"not" in stop_words

False

In [41]:
# removing the stop words in a list with a white space
stop_final=[' '.join(word for word in x.split() if word not in stop_words) for x in stop_splchar]

In [42]:
reviews_clean = stop_final
reviews_clean[:4]

['service worst pricing menu different bill give bill increased pricing even serving watermenu order need call times even non busy day',
 'really appreciate quality timing tried thattil kutti dosa addicted dosa really chutney really good money worth much better thattukada must try',
 'went friday night place surprisingly empty interesting menu almost fully made dosas bullseye dosa cheese masala dosa bullseye dosa really good egg perfectly cooked half boiled state masala cheese masala good cheese bit chewy liking chutney good sambar average dishes reasonably priced',
 'decent place serving good food ordered chilli fish chicken pork sizzler everything tasted good pork could slightly better cooked tried beverages sweet']

In [43]:
reviews1.rating

0        1.0
1        5.0
2        4.0
3        4.0
4        5.0
        ... 
27743    4.0
27744    4.0
27745    5.0
27746    5.0
27747    3.0
Name: rating, Length: 27748, dtype: float64

In [44]:
len(reviews_clean)

27748

## Splitting train and test set

Separate X and Y perform train test split, 70-30

In [45]:
X = reviews_clean
y = reviews1.rating

Train test split

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30,
                                                   random_state=42)

In [47]:
len(X_train), len(X_test)

(19423, 8325)

In [48]:
X_train[:2]

['great place weekend brunchdrinks available reasonable price variety food options available veg nonveg people',
 'came enjoy sushi ordered rainbow maki sriracha maki along salad amazing wanted try sushis wallet said no place bit pricey dessert disappointing though vietnamese banana cake supposed come candied banana came cut banana caramel replaced scorched service also poor ask thrice cheque ambience good seating comfortable mixed experience']

## Document term matrix using TfIdf

The term **tf–idf** stands for **term frequency–inverse document frequency**. It is a mathematical statistic that is planned to reflect how significant a word is to a record in a collection or corpus. 

There are some potential problems which might arise with the Bag of Words (N-Grams) model when it is used on large corpora. Since the feature vectors are based on absolute term frequencies, there might be some terms which occur frequently across all documents and these may tend to overshadow other terms in the feature set. The **TF-IDF** model tries to combat this issue by using a scaling or normalizing factor in its computation. This technique was developed for ranking results for queries in search engines and now it is an indispensable model in the world of information retrieval and NLP.

* **Term Frequency (tf)** - ***tf*** is used to measure the frequency of a word in the document. It gives us the ratio of the occurence of the word (w) in document (d) per the total number of words in the documents. 

        tf(w,d) = occurence of w in document d / total number of words in document d

* **Inverse Document Frequency (idf)** - **idf** is a measure of how common or rare a term is across the entire collection (corpus) of documents. So the point to note is that it’s common to all the documents. If the word is **common** and appears in many documents, the idf value (normalized) will approach **0** or else approach **1** if it’s **rare**. It is calculated by dividing the total number of documents in the corpus by the number of documents containing the word.
        
        idf(w,D) = log(total number of documents (N) in corpus D / number of documents containing w)

> **Term Frequency - Inverse Document Frequency (TF-IDF)** is the **product of term frequency and inverse document frequency. It gives more importance to the word that is rare in the corpus and common in a document.

        tfidf(w,d,D) = tf(w,d) x idf(w,D)

Here, we use `TfidfVectorizer` from `scikit-learn` to convert a collection of raw documents to a matrix of TF-IDF features.

> Here setting an argument, **max_features = 5000**, which means considering only the top 5000 features from a documents that is ordered by term frequency across the corpus.

We have a corpus of **19423** documents in a training set and **8325** documents in a testing set.

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
vectorizer = TfidfVectorizer(max_features = 5000)

In [51]:
# transform features
X_train_bow = vectorizer.fit_transform(X_train)
X_train_bow

<19423x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 484782 stored elements in Compressed Sparse Row format>

In [52]:
X_test_bow = vectorizer.transform(X_test)
X_test_bow

<8325x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 204812 stored elements in Compressed Sparse Row format>

In [53]:
print("Sparse Matrix shape of training set: ", X_train_bow.shape)
print("Sparse Matrix shape of training set: ", X_test_bow.shape)

Sparse Matrix shape of training set:  (19423, 5000)
Sparse Matrix shape of training set:  (8325, 5000)


In [54]:
pd.set_option('display.max_columns',None)

In [55]:
X_train_bow_features = X_train_bow.toarray()
X_train_bow_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
# to see the tf-idf based feature vectors for each of text documents that show scaled and normalized values
entire_doc = vectorizer.get_feature_names_out()
df = pd.DataFrame(np.round(X_train_bow_features, 2), columns=entire_doc)
df.sample(10)

aa  aalo  aaloo  aam  aameen  ability  able  abs  absence  absolute  \
11415  0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
4925   0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
5763   0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
15682  0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
14787  0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
3053   0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
16494  0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
9149   0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
7911   0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   
13412  0.0   0.0    0.0  0.0     0.0      0.0   0.0  0.0      0.0       0.0   

       absolutely   ac  accept  acceptable  accepted  accepting  accepts  \
11415         0.0  0.0     0.0         0.0       0.0        0.0      0.0   
4925          0.0  0.0     0.0         0.0       0.0        0.0      0.0   
5763          0.0  0.0     0.0         0.0       0.0        0.0      0.0   
15682         0.0  0.0     0.0         0.0       0.0        0.0      0.0   
14787         0.0  0.0     0.0         0.0       0.0        0.0      0.0   
3053          0.0  0.0     0.0         0.0       0.0        0.0      0.0   
16494         0.0  0.0     0.0         0.0       0.0        0.0      0.0   
9149          0.0  0.0     0.0         0.0       0.0        0.0      0.0   
7911          0.0  0.0     0.0         0.0       0.0        0.0      0.0   
13412         0.0  0.0     0.0         0.0       0.0        0.0      0.0   

       accessible  accidentally  accommodate  accommodating  accomodate  \
11415         0.0           0.0          0.0            0.0         0.0   
4925          0.0           0.0          0.0            0.0         0.0   
5763          0.0           0.0          0.0            0.0         0.0   
15682         0.0           0.0          0.0            0.0         0.0   
14787         0.0           0.0          0.0            0.0         0.0   
3053          0.0           0.0          0.0            0.0         0.0   
16494         0.0           0.0          0.0            0.0         0.0   
9149          0.0           0.0          0.0            0.0         0.0   
7911          0.0           0.0          0.0            0.0         0.0   
13412         0.0           0.0          0.0            0.0         0.0   

       accompanied  accompaniments  accompany  accompanying  accomplished  \
11415          0.0             0.0        0.0           0.0           0.0   
4925           0.0             0.0        0.0           0.0           0.0   
5763           0.0             0.0        0.0           0.0           0.0   
15682          0.0             0.0        0.0           0.0           0.0   
14787          0.0             0.0        0.0           0.0           0.0   
3053           0.0             0.0        0.0           0.0           0.0   
16494          0.0             0.0        0.0           0.0           0.0   
9149           0.0             0.0        0.0           0.0           0.0   
7911           0.0             0.0        0.0           0.0           0.0   
13412          0.0             0.0        0.0           0.0           0.0   

       according  accordingly  account  achari  acquired  across  acting  \
11415        0.0          0.0      0.0     0.0       0.0     0.0     0.0   
4925         0.0          0.0      0.0     0.0       0.0     0.0     0.0   
5763         0.0          0.0      0.0     0.0       0.0     0.0     0.0   
15682        0.0          0.0      0.0     0.0       0.0     0.0     0.0   
14787        0.0          0.0      0.0     0.0       0.0     0.0     0.0   
3053         0.0          0.0      0.0     0.0       0.0     0.0     0.0   
16494        0.0          0.0      0.0     0.0       0.0     0.0     0.0   
9149         0.0   

In [57]:
df.shape

(19423, 5000)

## Model building

Importing necessary libraries of `RandomForestRegressor` and `GradientBoostingRegressor` from  `ensemble` on `scikit-learn`.

In [58]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

### Performance on RandomForestRegressor

In [59]:
learner_rf = RandomForestRegressor(random_state=42)

In [60]:
learner_rf.fit(X_train_bow, y_train)

RandomForestRegressor(random_state=42)

In [61]:
y_train_preds_RF = learner_rf.predict(X_train_bow)

In [62]:
y_train_preds_RF

array([4.99 , 3.   , 3.   , ..., 5.   , 3.94 , 4.505])

In [64]:
from sklearn import metrics
# print result of MAE on train set
print("MAE for Random Forest train set: ", metrics.mean_absolute_error(y_train, y_train_preds_RF))

#print result of MSE on train set
print("MSE for Random Forest train set: ", metrics.mean_squared_error(y_train, y_train_preds_RF))

#print result of RMSE on train set
print("RMSE for Random Forest train set: ", np.sqrt(metrics.mean_squared_error(y_train, y_train_preds_RF)))

#print result of R^2 on train set
print("R^2 for Random Forest train set: ", metrics.r2_score(y_train, y_train_preds_RF))

MAE for Random Forest train set:  0.10598694654339542
MSE for Random Forest train set:  0.0537239647499873
RMSE for Random Forest train set:  0.23178430652222187
R^2 for Random Forest train set:  0.9672686292902974


In [65]:
y_test_preds_RF = learner_rf.predict(X_test_bow)

In [66]:
# print result of MAE on test set
print("MAE for Random Forest test set: ", metrics.mean_absolute_error(y_test, y_test_preds_RF))

#print result of MSE on test set
print("MSE for Random Forest test set: ", metrics.mean_squared_error(y_test, y_test_preds_RF))

#print result of RMSE on test set
print("RMSE for Random Forest test set: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_preds_RF)))

#print result of R^2 on test set
print("R^2 for Random Forest test set: ", metrics.r2_score(y_test, y_test_preds_RF))

MAE for Random Forest test set:  0.23909895059886216
MSE for Random Forest test set:  0.2261535871504567
RMSE for Random Forest test set:  0.47555608202446187
R^2 for Random Forest test set:  0.8646312455428723


In [67]:
# Model accuracy on train set
acc_train_RF = learner_rf.score(X_train_bow, y_train)
print("Train accuracy for Random Forest: ", acc_train_RF*100,"%")

Train accuracy for Random Forest:  96.72686292902975 %


In [68]:
# Model accuracy on test set
acc_test_RF = learner_rf.score(X_test_bow, y_test)
print("Test accuracy for Random Forest: ", acc_test_RF*100,"%")

Test accuracy for Random Forest:  86.46312455428723 %


## Increase the number of trees

In [69]:
learner_rf = RandomForestRegressor(random_state=42, n_estimators=200)

In [70]:
%%time
learner_rf.fit(X_train_bow, y_train)

CPU times: total: 23min 46s
Wall time: 24min 12s


RandomForestRegressor(n_estimators=200, random_state=42)

In [71]:
y_train_preds_RF = learner_rf.predict(X_train_bow)

In [72]:
y_train_preds_RF

array([4.985, 3.   , 3.   , ..., 5.   , 3.94 , 4.505])

In [73]:
y_test_preds_RF = learner_rf.predict(X_test_bow)
y_test_preds_RF

array([4.    , 3.21  , 4.3425, ..., 3.875 , 3.015 , 4.    ])

In [74]:
# print result of MAE on train set
print("MAE for Random Forest train set: ", metrics.mean_absolute_error(y_train, y_train_preds_RF))

#print result of MSE on train set
print("MSE for Random Forest train set: ", metrics.mean_squared_error(y_train, y_train_preds_RF))

#print result of RMSE on train set
print("RMSE for Random Forest train set: ", np.sqrt(metrics.mean_squared_error(y_train, y_train_preds_RF)))

#print result of R^2 on train set
print("R^2 for Random Forest train set: ", metrics.r2_score(y_train, y_train_preds_RF))

MAE for Random Forest train set:  0.10521729489658227
MSE for Random Forest train set:  0.052874236317988765
RMSE for Random Forest train set:  0.2299439851746263
R^2 for Random Forest train set:  0.9677863270521762


In [75]:
# print result of MAE on test set
print("MAE for Random Forest test set: ", metrics.mean_absolute_error(y_test, y_test_preds_RF))

#print result of MSE on test set
print("MSE for Random Forest test set: ", metrics.mean_squared_error(y_test, y_test_preds_RF))

#print result of RMSE on test set
print("RMSE for Random Forest test set: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_preds_RF)))

#print result of R^2 on test set
print("R^2 for Random Forest test set: ", metrics.r2_score(y_test, y_test_preds_RF))

MAE for Random Forest test set:  0.23802791222294004
MSE for Random Forest test set:  0.2248265772479292
RMSE for Random Forest test set:  0.4741588101553415
R^2 for Random Forest test set:  0.8654255538707695


In [76]:
# Model accuracy on train set
acc_train_RF = learner_rf.score(X_train_bow, y_train)
print("Train accuracy for Random Forest: ", acc_train_RF*100,"%")

Train accuracy for Random Forest:  96.77863270521762 %


In [77]:
# Model accuracy on test set
acc_test_RF = learner_rf.score(X_test_bow, y_test)
print("Test accuracy for Random Forest: ", acc_test_RF*100,"%")

Test accuracy for Random Forest:  86.54255538707694 %


## Hyper-parameter tuning

Hyperparameters are important parts of the ML model and can make the model gold or trash. Here we have discussed one of the popular hyperparameter tunning method i.e. using **Grid Search CV**. 

`GridSearchCV` performs an exhaustive search over specified parameter values for an estimator.

"class_weights" was one of the many hyperparameters to tune for the SVM

In [83]:
from sklearn.model_selection import GridSearchCV

Instantiate the learner with a random state

In [84]:
learner_rf = RandomForestRegressor(random_state = 42)

**Step 1:** Define a parameter space

In [85]:
# Create the parameter grid based on the results of random search
param_grid = {
    'max_features': [None, "sqrt", "log2", "auto"], # the number of features to consider when looking for the best split
    'max_depth': ['None', 10, 15, 20, 25], # Maximum number of levels in tree. Like a linspace function from numpy session
    'criterion': ["squared_error", "absolute_error", "poisson"], # what criteria to consider
    'n_estimators': [100, 200, 300, 400, 500] # Number of trees in random forest
    }

**Step 2:** Fit the model to find the best hyperparameters on training data, and select the scorer you want to select to optimise

In [86]:
# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator = learner_rf, param_grid = param_grid,
                          cv = 5, n_jobs = -1, verbose = 1, scoring = "neg_mean_squared_error" )

In [ ]:
grid_search_rf.fit(X_train_bow, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


In [ ]:
grid_search_rf.cv_results_

**Step 3:** Print the best obtained parameters

In [ ]:
grid_search_rf.best_estimator_

In [ ]:
grid_search_rf.best_params_

In [ ]:
grid_search_rf.best_score_

In [ ]:
grid_search_rf.score(X_test, Y_test)

### Performance on GradientBoostingRegressor

In [ ]:
learner_gb = GradientBoostingRegressor(random_state = 42)

In [ ]:
# Create the parameter grid based on the results of random search
param_grid_gb = {
    'max_features': [500, "sqrt", "log2", "auto"], # the number of features to consider when looking for the best split
    'max_depth': ['None', 10, 15, 20, 25], # Maximum number of levels in tree. Like a linspace function from numpy session
    'criterion': ["friedman_mse", "squared_error", "mse"], # what criteria to consider
    'loss': ["‘squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’"], # what loss to consider
    'learning_rate': [0.1, 0.01, 0.001] # Learning rate shrinks the contribution of each tree
    'n_estimators': [100, 200, 300, 400, 500] # Number of trees in random forest
}

In [ ]:
# Instantiate the grid search model
grid_search_gb = GridSearchCV(estimator = learner_gb, param_grid = param_grid_gb,
                          cv = 5, n_jobs = -1, verbose = 1, scoring = "neg_mean_squared_error" )

In [ ]:
grid_search_gb.fit(X_train_bow, y_train)

In [ ]:
grid_search_gb.cv_results_

In [ ]:
grid_search_gb.best_estimator_

In [ ]:
grid_search_gb.best_params_

In [ ]:
grid_search_gb.best_score_

In [ ]:
grid_search_gb.score(X_test, Y_test)

## Using the best estimator to make predictions on the test set

In [ ]:
y_train_pred_tuned = grid_search.best_estimator_.predict(X_train_bow)

In [ ]:
y_test_pred_tuned = grid_search.best_estimator_.predict(X_test_bow)

In [ ]:
mean_squared_error(y_train,y_train_pred)**0.5

In [ ]:
mean_squared_error(y_train,y_test_pred)**0.5

## Identifying mismatch cases

In [ ]:
res_df = pd.DataFrame({'review':X_test, 'rating':y_test, 'rating_pred':y_test_pred})

In [ ]:
res_df[(res_df.rating - res_df.rating_pred)>=2].shape

In [ ]:
res_df[(res_df.rating - res_df.rating_pred)>=2]